In [ ]:
import panel as pn
import numpy as np
import pandas as pd
import holoviews as hv
import squarify
import colorcet as cc

from cdapython import fetch_rows
from lumen.ai import ExplorerUI

pn.extension('filedropper') #"vega", "filedropper", #"vega", "filedropper", 
hv.extension("bokeh")

In [ ]:
def get_cda_table(table='diagnosis'):
    print(f"Fetching '{table}' table from CDA")
    df = fetch_rows(f'{table}')
    print(f"CDA '{table}' shape: {df.shape}")
    return df

In [ ]:
df = get_cda_table(table='diagnosis')
col_list = list(df.columns)

In [ ]:
col_list = [
    "primary_diagnosis",
    "morphology",
    "stage",
    "grade",
    "method_of_diagnosis",
]

In [ ]:
df.head()

In [ ]:
def build_treemap(
    df: pd.DataFrame,
    column_name: str,
    width=250,
    height=250
) -> hv.Element:
    """
    Create a treemap for a column using squarify + HoloViews (Rectangles).

    - Missing data => 'NA'.
    - Rectangle area => frequency of each distinct value.
    """
    col_vals = df[column_name].fillna("NA").astype(str)
    counts = col_vals.value_counts(dropna=False)
    if counts.empty:
        return hv.Text(0.5, 0.5, f"No data for '{column_name}'").opts(
            width=width, height=height
        )

    data_df = pd.DataFrame({"value": counts.index, "freq": counts.values})
    sizes = data_df["freq"].tolist()
    normed = squarify.normalize_sizes(sizes, width, height)
    rects = squarify.squarify(normed, 0, 0, width, height)

    rectangle_data = []
    for i, r in enumerate(rects):
        x1, y1 = r["x"], r["y"]
        x2, y2 = (r["x"] + r["dx"]), (r["y"] + r["dy"])

        label = data_df.iloc[i]["value"]
        freq = str(data_df.iloc[i]["freq"])
        rectangle_data.append((x1, y1, x2, y2, label, freq))

    treemap = hv.Rectangles(
        rectangle_data,
        kdims=["x1", "y1", "x2", "y2"],
        vdims=["freq", "label"]
    ).opts(
        width=width,
        height=height,
        invert_yaxis=True,
        color="freq",
        cmap=cc.b_glasbey_hv,
        frame_height=height,
        frame_width=width,
        tools=["hover"],
        xaxis="bare",
        yaxis="bare",
        padding=0,
        apply_hard_bounds=True,
        line_color="white",
        line_width=0.5,
        line_alpha=0.5,
        show_legend=False,
        # legend_position='right',
    ).relabel(f"Treemap: {column_name}")

    return treemap

treemaps = []
for c in col_list:
    treemaps.append(build_treemap(df, c,))
treemap_layout = hv.Layout(treemaps).opts(shared_axes=False).cols(2)

In [ ]:
treemap_layout

In [ ]:
cohexplorer = ExplorerUI()
combined = pn.Row(
    treemap_layout, cohexplorer._main,
)

# combined.servable()

template = pn.template.FastListTemplate(
        title="HoloViz Cohort Explorer Mockup",
        main=[combined],
        header_background="#445566",
        accent_base_color="#6688aa",
    ).servable() 